### ResNet50 Trained

### Initialize Notebook & packages

In [1]:
import helper as hp 
hp.initialize_notebook() # initialize with GPU enabled  
# hp.initialize_notebook(False) # to disable GPU 

2024-12-13 22:43:17.608053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 22:43:17.622750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 22:43:17.627368: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 22:43:17.639013: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU enabled. Checking for available GPUs...
1 Physical GPUs, 1 Logical GPUs

Verifying TensorFlow and PyTorch CUDA setup...
TensorFlow version: 2.17.0
Built with CUDA: True
Num GPUs Available: 1

Keras version: 3.6.0

End checks and initialization.


I0000 00:00:1734129799.567599  630556 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734129799.612493  630556 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734129799.615967  630556 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734129799.621336  630556 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
# Import standard libraries
import os
import sys
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import keras
# Import DL libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import layers, models, Model, Input
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.callbacks import TensorBoard 

# Suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import ML libraries
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

# Import image libraries
from skimage import transform
import tifffile as tiff

# Appends current working dir
current_path = os.getcwd()
sys.path.append(current_path)

# Import custom preprocessing class
from imc_preprocessing import IMCPreprocessor

# Import Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

#### Methods

In [3]:
# Preprocessing (if needed)
def preprocessing(image, transpose=True, normalize=True) -> np.ndarray:
    if transpose:
        return np.transpose(image, (1, 2, 0))
    if normalize:
        return IMCPreprocessor.normalize_multichannel_image(image)

# Load images
def load_image(image_path) -> np.ndarray:
    image = tiff.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    return image


# Define a function to create a list of images from files within a folder 
def image_list(image_dir):
    # List all files in the directory
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]  
    # Initialize a list to store the images
    images = []
    # Loop through each file and read the image
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        image = load_image(image_path)
        images.append(image)        
    return images 

# Converting to one hot
def convert_to_one_hot(y, classes):
    return np.eye(classes)[y]

### Data 

#### Preprocessing and Other

In [4]:
images_dir = '/home/jupyter-luvogt/Final_Project_LR/IMC_images' 
metadata_dir = '/home/jupyter-luvogt/Final_Project_LR/metadata.csv' 
panel_dir = '/home/jupyter-luvogt/Final_Project_LR/panel.csv' 
os.listdir(images_dir)[:5] # Get first five images

# Load images
images = image_list(images_dir)
images = np.array(images)

# load labels
metadata = pd.read_csv(metadata_dir)
PDL1_score = metadata["PDL1_score"]

# Shape PDL1
PDL1_score = PDL1_score.tolist()
PDL1_score = np.array(PDL1_score)

# Transpose and Normalize images
images_preproc = [preprocessing(i, transpose = True, normalize = False) for i in images]
images_preproc = [preprocessing(i, transpose = False, normalize = True) for i in images_preproc]
images_preproc = np.array(images_preproc)

# Extract channel information
panel_df = pd.read_csv(panel_dir)
channel_names = dict(zip(panel_df['clean_target'].to_list(), panel_df['channel'].to_list()))

# Filter out Xe131, Xe134 and Ba138 = Noise channels (OPTIONAL) 
channel_names_new = [x for x in list(channel_names.values()) if x not in ["Xe131", "Xe134", "Ba138"]]
images_preproc_drop = [IMCPreprocessor.drop_channels(i, channel_names_new, list(channel_names.values()))[0] for i in images_preproc]
images_preproc_drop = np.array(images_preproc_drop)

### ResNet50 Model: Trained directly here

#### ResNet50 Model: 3 Channels

Approach: Select biological relevant channels that correspond or are associated with PDL1 

In [5]:
# Choose biological relevant channels
channel_names_new = ["Gd160", "Eu153", "Gd155"]
images_preproc_drop_3 = [IMCPreprocessor.drop_channels(i, channel_names_new, list(channel_names.values()))[0] for i in images_preproc]
images_preproc_drop_3 = np.array(images_preproc_drop_3)

channels_preproc_drop_3 = channel_names_new

Create unbalanced (but with stratified) training, validation and test set

In [6]:
random_seed = 56
X = images_preproc_drop_3
y = PDL1_score
train_size = 0.6
val_size = 0.2
test_size = 0.2


# Create a StratifiedShuffleSplit for train/test split
sss_train_test = StratifiedShuffleSplit(n_splits=1, test_size=(val_size + test_size), random_state=random_seed)

# First split: Train and remaining (validation + test)
for train_index, remaining_index in sss_train_test.split(X, y):
    X_train, X_remaining = X[train_index], X[remaining_index]
    y_train, y_remaining = y[train_index], y[remaining_index]

# Create a StratifiedShuffleSplit for validation/test split on remaining data
sss_val_test = StratifiedShuffleSplit(n_splits=1, test_size=test_size / (val_size + test_size), random_state=random_seed)

# Second split: Validation and Test
for val_index, test_index in sss_val_test.split(X_remaining, y_remaining):
    X_val, X_test = X_remaining[val_index], X_remaining[test_index]
    y_val, y_test = y_remaining[val_index], y_remaining[test_index]


Downsampling

Define CONSTANTS and do One Hot Encoding

In [8]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
# from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf
CLASSES = 2
y_train_one_hot = convert_to_one_hot(y_train, CLASSES)
y_test_one_hot = convert_to_one_hot(y_test, CLASSES)
y_val_one_hot = convert_to_one_hot(y_val, CLASSES)

Define Blocks and ResNet Model

Build Model

In [11]:
# Build Pre Trained ResNet50
from keras.applications import ResNet50

resnet50 = ResNet50(weights="imagenet", include_top = True) # Include fully connected layers
# Replacing the final classification Layer
num_classes = 2
x = resnet50.layers[-2].output # Accessing the final classification layer
new_output = Dense(num_classes, activation = "softmax")(x)
# for layer in resnet50.layers:
#     layer.trainable = False
# new_output.trainable = True
resnet_new = Model(inputs = resnet50.input, outputs = new_output)



# Compile Model 
l_rate = 1e-3
opt = keras.optimizers.Adam(learning_rate=l_rate)

resnet_new.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# # Apply TensorBoard
# # define the logs folder 
# log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet50_3Channels" + time.strftime("%Y%m%d-%H%M%S"))
# # Define TensorBoard Callback
# tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train Model 
batch = 32
epochs = 10
start_time = time.time()

history_ResNet50_trained = resnet_new.fit(X_train, y_train_one_hot, 
                                      epochs = epochs, batch_size = batch, 
                                      validation_data = (X_val, y_val_one_hot))
                                      # callbacks=tb_callback)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

Epoch 1/10


I0000 00:00:1734129951.949765  630641 service.cc:146] XLA service 0x7f7e240038b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734129951.949798  630641 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-12-13 22:45:52.668152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-13 22:45:55.344978: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300
2024-12-13 22:46:00.112896: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 18.25GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
I0000 00:00:1734129976.726975  630641 device_compiler.h:188] Compiled cluster using XLA!  This line 

19/19 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.5472 - loss: 1.0097 - val_accuracy: 0.6396 - val_loss: 52.1065
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 260ms/step - accuracy: 0.6221 - loss: 0.7164 - val_accuracy: 0.6396 - val_loss: 153.5427
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 263ms/step - accuracy: 0.6170 - loss: 0.6549 - val_accuracy: 0.3604 - val_loss: 55.2975
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 265ms/step - accuracy: 0.6489 - loss: 0.6663 - val_accuracy: 0.3604 - val_loss: 1.4350
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 266ms/step - accuracy: 0.6306 - loss: 0.6343 - val_accuracy: 0.6396 - val_loss: 0.6663
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - accuracy: 0.6619 - loss: 0.6037 - val_accuracy: 0.6396 - val_loss: 4.6068
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 270ms/step - accuracy: 0.7389 - loss: 0.5359 - val_accuracy: 0.6396 - val_loss: 9.7457
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 273ms/step - accuracy: 0.8870 - loss: 0.3028 - val_accuracy: 0.6396 - v

In [38]:
resnet_new.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 461ms/step


array([[0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.74560446, 0.25439557],
       [0.

INTERESTING: validation accuracy constantly 0.6396 (or 0.3604) indicates that model only predicts PDL1 == 0 as this is the distribution in our unbalanced validation set

Indication: Overfitting

Possible sources for lack of generalization: 

    - Lack of Regularization
    - Unbalanced Dataset
    - Overly complex model
    - Add more channels because 3 channels don't contain enough information for generalization? 

### ResNet50 Model: 3 Channels & L2 Regularization

In [22]:
# Build Pre Trained ResNet50 with L2 Regularization
from keras.applications import ResNet50
from keras.regularizers import l2

resnet50 = ResNet50(weights="imagenet", include_top = True) # Include fully connected layers
# Replacing the final classification Layer
num_classes = 2
x = resnet50.layers[-2].output # Accessing the final classification layer
new_output = Dense(num_classes, activation = "softmax", kernel_regularizer=l2(0.01))(x)
# for layer in resnet50.layers:
#     layer.trainable = False
# new_output.trainable = True
resnet_new = Model(inputs = resnet50.input, outputs = new_output)



# Compile Model 
l_rate = 1e-4
opt = keras.optimizers.Adam(learning_rate=l_rate)

resnet_new.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# # Apply TensorBoard
# # define the logs folder 
# log_dir = os.path.join("logs_ResNet50", "fit", "model_ResNet50_3Channels" + time.strftime("%Y%m%d-%H%M%S"))
# # Define TensorBoard Callback
# tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train Model 
batch = 32
epochs = 10
start_time = time.time()

history_ResNet50_trained = resnet_new.fit(X_train, y_train_one_hot, 
                                      epochs = epochs, batch_size = batch, 
                                      validation_data = (X_val, y_val_one_hot))
                                      # callbacks=tb_callback)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nElapsed time: {elapsed_time} seconds")

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.5646 - loss: 0.7581 - val_accuracy: 0.6396 - val_loss: 0.6993
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 263ms/step - accuracy: 0.9764 - loss: 0.3228 - val_accuracy: 0.6396 - val_loss: 0.7026
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 265ms/step - accuracy: 1.0000 - loss: 0.1097 - val_accuracy: 0.6396 - val_loss: 0.7171
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 269ms/step - accuracy: 1.0000 - loss: 0.0590 - val_accuracy: 0.6396 - val_loss: 0.7530
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 270ms/step - accuracy: 1.0000 - loss: 0.0476 - val_accuracy: 0.6396 - val_loss: 0.8173
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 271ms/step - accuracy: 1.0000 - loss: 0.0458 - val_accuracy: 0.6396 - val_loss: 0.9357
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 273ms/step - accuracy: 1.0000 - loss: 0.0440 - val_accuracy: 0.6396 - val_loss: 0.9181
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 275ms/step - accuracy: 1.0000 - loss: 0.0431 - val_accuracy: 0.63

#### COMMENTS

Problem persists when Regularization added to last classification layer --> use manually constructed ResNet to experiment with Regularization

In [ ]:
from keras.applications import ResNet20